In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import openai
import io
import pandas as pd
import tqdm
import sys
import json
import ast
from datetime import datetime


sys.path.append('../hot_links')
# TODO Something like this in dataset.py
# from hot_links.config import PROCESSED_DATA_DIR, RAW_DATA_DIR

from hot_links.vector_db_utils import *


2025-08-29 10:08:32.148 | INFO     | hot_links.config:<module>:11 - PROJ_ROOT path is: /workspaces/hot-links


In [46]:
test_df = pd.read_csv('../data/processed/final_test.csv')

In [47]:
test_df.head()

,links,content,url,date
0,[{'href': 'https://www.theguardian.com/world/s...,Donald Trump hasrenewed a threat to impose san...,https://www.theguardian.com/world/2025/aug/23/...,1755907200
1,[{'href': 'https://www.theguardian.com/culture...,Women’s groups have said a high court judgment...,https://www.theguardian.com/culture/2025/aug/2...,1755907200
2,[{'href': 'https://www.theguardian.com/comment...,Lots of people told me to not study languages ...,https://www.theguardian.com/commentisfree/2025...,1755907200
3,[{'href': 'https://www.theguardian.com/technol...,There are growing fears of an imminent stock m...,https://www.theguardian.com/technology/2025/au...,1755907200
4,[{'href': 'https://www.theguardian.com/film/fi...,"It is clear, as we look forward to this autumn...",https://www.theguardian.com/film/2025/aug/23/c...,1755907200


In [48]:
test_df.head(1)['links'].values[0]

"[{'href': 'https://www.theguardian.com/world/series/russia-ukraine-war-at-a-glance', 'link': 'Russia-Ukraine war at a glance'}, {'href': 'https://www.theguardian.com/world/ukraine', 'link': 'Ukraine'}, {'href': 'https://www.theguardian.com/world/2025/aug/23/trump-again-mulls-russia-sanctions-as-ukraine-peace-remains-elusive-one-week-after-alaska-summit', 'link': 'renewed a threat to impose sanctions on Russia'}, {'href': 'https://www.theguardian.com/world/2025/aug/22/ukraine-attacks-pipeline-that-sends-russian-oil-to-hungary-and-slovakia', 'link': 'Deborah Cole reports'}, {'href': 'https://www.theguardian.com/world/slovakia', 'link': 'Slovakia'}, {'href': 'https://www.theguardian.com/world/2025/aug/22/prigozhin-knew-he-was-doomed-after-failed-rebellion-says-mother', 'link': 'Pjotr Sauer reports'}, {'href': 'https://www.theguardian.com/world/2023/jan/24/yevgeny-prigozhin-the-hotdog-seller-who-rose-to-the-top-of-putin-war-machine-wagner-group', 'link': 'founder'}, {'href': 'https://www.

In [60]:
test_df['links'] = test_df['links'].apply(ast.literal_eval)

In [61]:
for link in test_df.head(1)['links'].values[0]:
    print(link)


{'href': 'https://www.theguardian.com/world/series/russia-ukraine-war-at-a-glance', 'link': 'Russia-Ukraine war at a glance'}
{'href': 'https://www.theguardian.com/world/ukraine', 'link': 'Ukraine'}
{'href': 'https://www.theguardian.com/world/2025/aug/23/trump-again-mulls-russia-sanctions-as-ukraine-peace-remains-elusive-one-week-after-alaska-summit', 'link': 'renewed a threat to impose sanctions on Russia'}
{'href': 'https://www.theguardian.com/world/2025/aug/22/ukraine-attacks-pipeline-that-sends-russian-oil-to-hungary-and-slovakia', 'link': 'Deborah Cole reports'}
{'href': 'https://www.theguardian.com/world/slovakia', 'link': 'Slovakia'}
{'href': 'https://www.theguardian.com/world/2025/aug/22/prigozhin-knew-he-was-doomed-after-failed-rebellion-says-mother', 'link': 'Pjotr Sauer reports'}
{'href': 'https://www.theguardian.com/world/2023/jan/24/yevgeny-prigozhin-the-hotdog-seller-who-rose-to-the-top-of-putin-war-machine-wagner-group', 'link': 'founder'}
{'href': 'https://www.theguardi

In [8]:
vector_store_id = get_vector_store_id()

vs_68aee23606108191acbb1b3f0dfc4f69 hot_links


In [9]:
vector_store_id

'vs_68aee23606108191acbb1b3f0dfc4f69'

In [10]:
test_df.iloc[3,]['links']

[{'href': 'https://www.theguardian.com/australia-news/new-south-wales-politics',
  'link': 'New South Wales politics'},
 {'href': 'https://www.theguardian.com/profile/michael-mcgowan',
  'link': 'Michael McGowan'},
 {'href': 'https://www.theguardian.com/technology/ng-interactive/2018/may/15/the-guardian-app?CMP=cvau_sfl',
  'link': 'free news app'},
 {'href': 'https://www.theguardian.com/world/guardian-australia-morning-mail/2014/jun/24/-sp-guardian-australias-morning-mail-subscribe-by-email?CMP=cvau_sfl',
  'link': 'morning email briefing'},
 {'href': 'https://www.theguardian.com/australia-news/series/full-story?CMP=cvau_sfl',
  'link': 'daily news podcast'},
 {'href': 'https://www.theguardian.com/australia-news/new-south-wales',
  'link': 'New South Wales'},
 {'href': 'https://www.theguardian.com/australia-news/commentisfree/2022/jun/24/john-barilaro-new-york-job-us-trade-commissioner-appointment',
  'link': 'Anthony Whealy QC wrote last week'},
 {'href': 'https://www.theguardian.com

In [11]:
for i, link in enumerate(test_df.iloc[3,]['links']):
    print(i, link['link'],link['href'])

0 New South Wales politics https://www.theguardian.com/australia-news/new-south-wales-politics
1 Michael McGowan https://www.theguardian.com/profile/michael-mcgowan
2 free news app https://www.theguardian.com/technology/ng-interactive/2018/may/15/the-guardian-app?CMP=cvau_sfl
3 morning email briefing https://www.theguardian.com/world/guardian-australia-morning-mail/2014/jun/24/-sp-guardian-australias-morning-mail-subscribe-by-email?CMP=cvau_sfl
4 daily news podcast https://www.theguardian.com/australia-news/series/full-story?CMP=cvau_sfl
5 New South Wales https://www.theguardian.com/australia-news/new-south-wales
6 Anthony Whealy QC wrote last week https://www.theguardian.com/australia-news/commentisfree/2022/jun/24/john-barilaro-new-york-job-us-trade-commissioner-appointment
7 John Barilaro withdraws from New York trade role due to ‘media attention’Read more https://www.theguardian.com/australia-news/2022/jun/30/john-barilaro-withdraws-from-new-york-trade-role-due-to-media-attention
8

In [63]:
test_article = test_df.iloc[5,]

In [64]:
test_article_all_links = [d['href'] for d in test_article['links']]

In [30]:
test_article_link = test_article['links'][2]

In [66]:
test_article_date = test_article['date']

In [15]:
query = '''
Which document best matches the article text below?
--------
{:s}
'''.format(test_article['content'])

In [31]:
print(test_article_link['link'])
print(test_article_link['href'])

poorly insulated homes
https://www.theguardian.com/money/2022/jan/14/fund-home-insulation-and-heat-pumps-for-people-on-low-incomes-pm-urged


In [67]:
query = '''
If I wanted to add a hyperlink to a document at this point '{:s}' in this document >{:s}<, which document would be most relevant and appropriate?
'''.format(test_article_link['link'],test_article['content'])

### RAG Query

In [18]:
results = query_db(vector_store_id=vector_store_id, query_string = query, time_stamp = datetime.now().timestamp())

In [19]:
print(results.output[1].content[0].text)

To add a hyperlink to the document at the specific point "Ayres said Barilaro had sent him a text asking about 'the status' of the New York job," you may find the article containing this quote on the Guardian's website. Here’s the relevant link you can use:

- Ayres regarding Barilaro's text about the New York job: [Guardian Article](https://www.theguardian.com/australia-news/2022/feb/22/nsw-transport-minister-went-to-bed-before-train-shutdown-decision-was-made)

You can format it in your document as follows:

> Ayres said Barilaro had sent him a text asking about “the status” of the [New York job](https://www.theguardian.com/australia-news/2022/feb/22/nsw-transport-minister-went-to-bed-before-train-shutdown-decision-was-made).

This hyperlink directs readers to the source of the information and provides context regarding the statement made by Ayres.


### Semantic search query

In [73]:
results = search_db(vector_store_id=vector_store_id, query=query[:4096], time_stamp = test_article_date)
# Limit of 4096 chars

In [74]:
print(query)


If I wanted to add a hyperlink to a document at this point 'do the “honourable thing” and quit' in this document >Lyle Menendez was denied parole for his role in the 1989 killings of his parents on Friday, just a day after the California parole boarddenied the release of his brotherErik.California governor Gavin Newsom will have the final say in whether or not the 57-year-old will be released.California parole board denies release of Erik MenendezRead moreLyle and Erik Menendez have spentnearly 30 years behind barsfor the murders of their parents. The brothers, who were 18 and 21 at the time of the killings, have said they fatally shot José and Kitty Menendez after years of molestation by their father. They were sentenced to life in prison in 1996, after prosecutors argued they committed the violence in order to receive a multimillion-dollar inheritance.The case has long fascinated the public and drawn international attention. Their first trial was televised by Court TV.But their stor

In [75]:
def parse_results(results, v = True):
    
    n_article_match = 0
    n_link_match = 0
    matched_at_k = []
    
    for r in results:
        if v:
            print('{:.2f}'.format(r.score))
            print(r.filename)
            #print(r.content)
        if r.filename.split('.txt')[0] in test_article_all_links:
            if v:
                print('Linked in article')
            n_article_match += 1
        if r.filename.split('.txt')[0] == test_article_link['href']:
            if v:
                print('Matches text')
                matched_at_k.append(r)
                n_link_match += 1
        if v:print()
    return n_article_match, n_link_match, matched_at_k

In [76]:
parse_results(results, v = False)

(0, 0, [])

## Put it in a loop

In [77]:
test_article

links      [{'href': 'https://www.theguardian.com/us-news...
content    Lyle Menendez was denied parole for his role i...
url        https://www.theguardian.com/us-news/2025/aug/2...
date                                              1755907200
Name: 5, dtype: object

In [ ]:
test_article = test_df.iloc[3,]
# Get one article from test set

test_article_all_links = [d['href'] for d in test_article['links']]
# Get all the links from that article

print('Article has {:d} links'.format(len(test_article_all_links)))

test_article_date = test_article['date']
# Get the article date to limit search to older articles

for test_article_link in tqdm.tqdm(test_article['links']):
    # Loop over the links in that article
    
    query = '''
    If I wanted to add a hyperlink to a document at this point '{:s}' in this document >{:s}<, which document would be most relevant and appropriate?
    '''.format(test_article_link['link'],test_article['content'])
    # Make the query

    results = search_db(vector_store_id=vector_store_id, query=query[:4096],time_stamp=datetime.now().timestamp())
    # Make the semantic search

    n_article_match, n_link_match, matched_at_k = parse_results(results, v = False)
    print(n_article_match, n_link_match, matched_at_k)

Article has 13 links


  0%|          | 0/13 [00:00<?, ?it/s]

  8%|▊         | 1/13 [00:01<00:17,  1.42s/it]

0 0 []


 15%|█▌        | 2/13 [00:03<00:22,  2.03s/it]

...

0 0 []


 92%|█████████▏| 12/13 [00:22<00:01,  1.75s/it]

0 0 []


100%|██████████| 13/13 [00:24<00:00,  1.88s/it]

0 0 []
